In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

### 가짜 뉴스 탐지기

# 기본 데이터

# 데이터 전처리

In [2]:
import pandas as pd

fake_path = 'Fake.csv'
true_path = 'True.csv'
fake_df = pd.read_csv(fake_path)
true_df = pd.read_csv(true_path)

## 데이터 불러오기

In [3]:
print(fake_df.tail(3))

                                                   title  \
23478  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
23479  How to Blow $700 Million: Al Jazeera America F...   
23480  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text      subject  \
23478  Patrick Henningsen  21st Century WireRemember ...  Middle-east   
23479  21st Century Wire says Al Jazeera America will...  Middle-east   
23480  21st Century Wire says As 21WIRE predicted in ...  Middle-east   

                   date  
23478  January 15, 2016  
23479  January 14, 2016  
23480  January 12, 2016  


## fake_df의 데이터 확인

In [4]:
print(true_df.tail(3))

                                                   title  \
21414  Minsk cultural hub becomes haven from authorities   
21415  Vatican upbeat on possibility of Pope Francis ...   
21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text    subject  \
21414  MINSK (Reuters) - In the shadow of disused Sov...  worldnews   
21415  MOSCOW (Reuters) - Vatican Secretary of State ...  worldnews   
21416  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...  worldnews   

                   date  
21414  August 22, 2017   
21415  August 22, 2017   
21416  August 22, 2017   


## ture_df의 데이터 확인

In [5]:
print(true_df.tail(3))

                                                   title  \
21414  Minsk cultural hub becomes haven from authorities   
21415  Vatican upbeat on possibility of Pope Francis ...   
21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text    subject  \
21414  MINSK (Reuters) - In the shadow of disused Sov...  worldnews   
21415  MOSCOW (Reuters) - Vatican Secretary of State ...  worldnews   
21416  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...  worldnews   

                   date  
21414  August 22, 2017   
21415  August 22, 2017   
21416  August 22, 2017   


## 진짜 뉴스와 가짜 뉴스를 구분하기 위한 label 열 생성

In [6]:
fake_df['label'] = 1
true_df['label'] = 0

## fake_df, true_df를 result_df에 합쳐서 저장

In [7]:
result_df = pd.concat([fake_df, true_df], ignore_index=True)
result_df

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


## 확인

In [8]:
print(result_df[['title','label']].tail(3))

                                                   title  label
44895  Minsk cultural hub becomes haven from authorities      0
44896  Vatican upbeat on possibility of Pope Francis ...      0
44897  Indonesia to buy $1.14 billion worth of Russia...      0


## 새로운 열 message_length를 만들어주고 text, title의 문자 길이를 저장

In [9]:
result_df['message_length'] = result_df['text'].apply(len)
result_df['message_length'] += result_df['title'].apply(len)

In [10]:
print(result_df['message_length'].tail(3))

44895    1999
44896    1260
44897    1390
Name: message_length, dtype: int64


# 데이터 전처리2

In [11]:
import nltk
nltk.download('stopwords')
nltk.download("names")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/t16user3/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to /home/t16user3/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords
from nltk.corpus import names

In [13]:
all_names = set(names.words())

In [14]:
all_names_copy = list(all_names)
print(all_names_copy[:5])

['Blakeley', 'Reube', 'Avery', 'Davida', 'Theodore']


## clean text함수 적용

In [15]:
#clean_text 함수는 mess에 있는 string.punctuation, stop_words, all_names의 값들을 모두 제거해주고, 모든 대문자 알파벳을 소문자로 바꿔준다
import string
def clean_text(mess):
    stop_words = stopwords.words('english')
    del_punctuation = [char.lower() for char in mess if char not in string.punctuation]
    del_punctuation = ''.join(del_punctuation)

    return ' '.join([word for word in del_punctuation.split() if word.lower() not in (stop_words or all_names)])

In [16]:
stopwords.words('english')[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [17]:
result_df['clean_message'] = result_df['text'].apply(clean_text)
result_df['clean_message'] += result_df['title'].apply(clean_text)

## clean_message와 text를 비교해보면 불필요한 표현들이 없어진 것을 확인

In [18]:
print(result_df[['clean_message', 'text']].tail())

                                           clean_message  \
44893  brussels reuters nato allies tuesday welcomed ...   
44894  london reuters lexisnexis provider legal regul...   
44895  minsk reuters shadow disused sovietera factori...   
44896  moscow reuters vatican secretary state cardina...   
44897  jakarta reuters indonesia buy 11 sukhoi fighte...   

                                                    text  
44893  BRUSSELS (Reuters) - NATO allies on Tuesday we...  
44894  LONDON (Reuters) - LexisNexis, a provider of l...  
44895  MINSK (Reuters) - In the shadow of disused Sov...  
44896  MOSCOW (Reuters) - Vatican Secretary of State ...  
44897  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...  


# 끝

# Politics

In [19]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
valid = pd.read_csv('valid.csv')

In [20]:
train['abortion'].value_counts()

abortion
health-care                                                       381
taxes                                                             308
immigration                                                       253
elections                                                         252
education                                                         237
                                                                 ... 
pensions,retirement,state-budget,state-finances,unions,workers      1
foreign-policy,history,iraq                                         1
economy,health-care,stimulus                                        1
congress,economy                                                    1
florida,foreign-policy                                              1
Name: count, Length: 3827, dtype: int64

In [21]:
# import pandas as pd

# # .tsv 파일 읽기
# train = 'train.tsv'
# df = pd.read_csv(train, sep='\t')

# # 데이터 확인
# print(df.head())

# # .csv 파일로 저장
# csv_file = 'train.csv'
# df.to_csv(csv_file, index=False)

In [22]:
# import pandas as pd

# # .tsv 파일 읽기
# test = 'test.tsv'
# df = pd.read_csv(test, sep='\t')

# # 데이터 확인
# print(df.head())

# # .csv 파일로 저장
# csv_file = 'test.csv'
# df.to_csv(csv_file, index=False)

In [23]:
# import pandas as pd

# # .tsv 파일 읽기
# valid = 'valid.tsv'
# df = pd.read_csv(valid, sep='\t')

# # 데이터 확인
# print(df.head())

# # .csv 파일로 저장
# csv_file = 'valid.csv'
# df.to_csv(csv_file, index=False)

In [24]:
train['false'].value_counts()

false
half-true      2114
false          1994
mostly-true    1962
true           1676
barely-true    1654
pants-fire      839
Name: count, dtype: int64

In [25]:
test

,11972.json,true,Building a wall on the U.S.-Mexico border will take literally years.,immigration,rick-perry,Governor,Texas,republican,30,30.1,42,23,18,Radio interview
0,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference
1,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.
2,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show
3,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video
4,5962.json,true,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,7334.json,half-true,Says his budget provides the highest state fun...,education,rick-scott,Governor,Florida,republican,28,23,38,34,7,a news conference
1262,9788.json,barely-true,Ive been here almost every day.,"civil-rights,crime,criminal-justice",jay-nixon,Governor,Missouri,democrat,2,0,0,1,0,"on ABC's ""This Week"""
1263,10710.json,barely-true,"In the early 1980s, Sen. Edward Kennedy secret...","bipartisanship,congress,foreign-policy,history",mackubin-thomas-owens,"senior fellow, Foreign Policy Research Institute",Rhode Island,columnist,1,0,0,0,0,a commentary in The Providence Journal
1264,3186.json,barely-true,Says an EPA permit languished under Strickland...,"environment,government-efficiency",john-kasich,"Governor of Ohio as of Jan. 10, 2011",Ohio,republican,9,8,10,18,3,a news conference


In [26]:
new_df = train[['false', 'Says the Annies List political group supports third-trimester abortions on demand.']]
new_df.columns = ['label','subject']
new_df['category'] = 'politics'
# new_df
new_df = new_df[new_df['label'].isin(['true', 'mostly-true','half-true', 'barely-true', 'false', 'pants-fire'])]
# new_df
new_df.to_csv('train2.csv', index=False)
new_df = pd.read_csv('train2.csv')
new_df

/tmp/ipykernel_854560/3971478140.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['category'] = 'politics'


,label,subject,category
0,half-true,When did the decline of coal start? It started...,politics
1,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",politics
2,false,Health care reform legislation is likely to ma...,politics
3,half-true,The economic turnaround started at the end of ...,politics
4,true,The Chicago Bears have had more starting quart...,politics
...,...,...,...
10234,mostly-true,There are a larger number of shark attacks in ...,politics
10235,mostly-true,Democrats have now become the party of the [At...,politics
10236,half-true,Says an alternative to Social Security that op...,politics
10237,false,On lifting the U.S. Cuban embargo and allowing...,politics


In [27]:
new_df['label'].value_counts()

label
half-true      2114
false          1994
mostly-true    1962
true           1676
barely-true    1654
pants-fire      839
Name: count, dtype: int64

In [28]:
new_df2 = test[['true', 'Building a wall on the U.S.-Mexico border will take literally years.']]
new_df2.columns = ['label','subject']
new_df2['category'] = 'politics'
# new_df
new_df2 = new_df[new_df['label'].isin(['true', 'mostly-true','half-true', 'barely-true', 'false', 'pants-fire'])]
# new_df
new_df2.to_csv('test2.csv', index=False)
new_df2 = pd.read_csv('test2.csv')
new_df2

/tmp/ipykernel_854560/118203713.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df2['category'] = 'politics'


,label,subject,category
0,half-true,When did the decline of coal start? It started...,politics
1,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",politics
2,false,Health care reform legislation is likely to ma...,politics
3,half-true,The economic turnaround started at the end of ...,politics
4,true,The Chicago Bears have had more starting quart...,politics
...,...,...,...
10234,mostly-true,There are a larger number of shark attacks in ...,politics
10235,mostly-true,Democrats have now become the party of the [At...,politics
10236,half-true,Says an alternative to Social Security that op...,politics
10237,false,On lifting the U.S. Cuban embargo and allowing...,politics


In [29]:
new_df3 = valid[['barely-true', 'We have less Americans working now than in the 70s.']]
new_df3.columns = ['label','subject']
new_df3['category'] = 'politics'
# new_df
new_df3 = new_df[new_df['label'].isin(['true', 'mostly-true','half-true', 'barely-true', 'false', 'pants-fire'])]
# new_df
new_df3.to_csv('valid2.csv', index=False)
new_df3 = pd.read_csv('valid2.csv')
new_df3

/tmp/ipykernel_854560/1882940416.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df3['category'] = 'politics'


,label,subject,category
0,half-true,When did the decline of coal start? It started...,politics
1,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",politics
2,false,Health care reform legislation is likely to ma...,politics
3,half-true,The economic turnaround started at the end of ...,politics
4,true,The Chicago Bears have had more starting quart...,politics
...,...,...,...
10234,mostly-true,There are a larger number of shark attacks in ...,politics
10235,mostly-true,Democrats have now become the party of the [At...,politics
10236,half-true,Says an alternative to Social Security that op...,politics
10237,false,On lifting the U.S. Cuban embargo and allowing...,politics


In [30]:
result = pd.concat([new_df, new_df2, new_df3], ignore_index=True)
result

,label,subject,category
0,half-true,When did the decline of coal start? It started...,politics
1,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",politics
2,false,Health care reform legislation is likely to ma...,politics
3,half-true,The economic turnaround started at the end of ...,politics
4,true,The Chicago Bears have had more starting quart...,politics
...,...,...,...
30712,mostly-true,There are a larger number of shark attacks in ...,politics
30713,mostly-true,Democrats have now become the party of the [At...,politics
30714,half-true,Says an alternative to Social Security that op...,politics
30715,false,On lifting the U.S. Cuban embargo and allowing...,politics


In [31]:
# function to convert 6 classes of label into 2 classes
def classLabel(dataset):
    for index in range(len(dataset['label'])):
        if(dataset.iloc[index]['label'] == 'true' or dataset.iloc[index]['label'] == 'mostly-true' or dataset.iloc[index]['label'] == 'half-true'):
            dataset.iloc[index]['label'] = 'true'
      
        if(dataset.iloc[index]['label'] == 'barely-true' or dataset.iloc[index]['label'] == 'false' or dataset.iloc[index]['label'] == 'pants-fire'):
            dataset.iloc[index]['label'] = 'false'
      
    return dataset

In [32]:
# Applying the function to the 'result' DataFrame
result = classLabel(result)
result

,label,subject,category
0,true,When did the decline of coal start? It started...,politics
1,true,"Hillary Clinton agrees with John McCain ""by vo...",politics
2,false,Health care reform legislation is likely to ma...,politics
3,true,The economic turnaround started at the end of ...,politics
4,true,The Chicago Bears have had more starting quart...,politics
...,...,...,...
30712,true,There are a larger number of shark attacks in ...,politics
30713,true,Democrats have now become the party of the [At...,politics
30714,true,Says an alternative to Social Security that op...,politics
30715,false,On lifting the U.S. Cuban embargo and allowing...,politics


In [33]:
result['label'] = result['label'].map({'false': 1, 'true': 0})
result.to_csv('result.csv', index=False)
result = pd.read_csv('result.csv')
result

,label,subject,category
0,0,When did the decline of coal start? It started...,politics
1,0,"Hillary Clinton agrees with John McCain ""by vo...",politics
2,1,Health care reform legislation is likely to ma...,politics
3,0,The economic turnaround started at the end of ...,politics
4,0,The Chicago Bears have had more starting quart...,politics
...,...,...,...
30712,0,There are a larger number of shark attacks in ...,politics
30713,0,Democrats have now become the party of the [At...,politics
30714,0,Says an alternative to Social Security that op...,politics
30715,1,On lifting the U.S. Cuban embargo and allowing...,politics


In [34]:
result['clean_message'] = result['subject'].apply(clean_text)
result

,label,subject,category,clean_message
0,0,When did the decline of coal start? It started...,politics,decline coal start started natural gas took st...
1,0,"Hillary Clinton agrees with John McCain ""by vo...",politics,hillary clinton agrees john mccain voting give...
2,1,Health care reform legislation is likely to ma...,politics,health care reform legislation likely mandate ...
3,0,The economic turnaround started at the end of ...,politics,economic turnaround started end term
4,0,The Chicago Bears have had more starting quart...,politics,chicago bears starting quarterbacks last 10 ye...
...,...,...,...,...
30712,0,There are a larger number of shark attacks in ...,politics,larger number shark attacks florida cases vote...
30713,0,Democrats have now become the party of the [At...,politics,democrats become party atlanta metro area blacks
30714,0,Says an alternative to Social Security that op...,politics,says alternative social security operates galv...
30715,1,On lifting the U.S. Cuban embargo and allowing...,politics,lifting us cuban embargo allowing travel cuba


# 끝

# Covid

In [35]:
cv_train = pd.read_csv('Constraint_Train.csv')
cv_val = pd.read_csv('Constraint_Val.csv')

In [36]:
result_df3 = pd.concat([cv_train, cv_val], ignore_index=True)
result_df3

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
...,...,...,...
8555,2136,Donald Trump wrongly claimed that New Zealand ...,fake
8556,2137,Current understanding is #COVID19 spreads most...,real
8557,2138,Nothing screams “I am sat around doing fuck al...,fake
8558,2139,Birx says COVID-19 outbreak not under control ...,fake


In [37]:
result_df3['category'] = 'medical'

In [38]:
result_df3['label'] = result_df3['label'].map({'fake': 1, 'real': 0})
result_df3.to_csv('result3.csv', index=False)
result_df3 = pd.read_csv('result3.csv')
result_df3

,id,tweet,label,category
0,1,The CDC currently reports 99031 deaths. In gen...,0,medical
1,2,States reported 1121 deaths a small rise from ...,0,medical
2,3,Politically Correct Woman (Almost) Uses Pandem...,1,medical
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,0,medical
4,5,Populous states can generate large case counts...,0,medical
...,...,...,...,...
8555,2136,Donald Trump wrongly claimed that New Zealand ...,1,medical
8556,2137,Current understanding is #COVID19 spreads most...,0,medical
8557,2138,Nothing screams “I am sat around doing fuck al...,1,medical
8558,2139,Birx says COVID-19 outbreak not under control ...,1,medical


In [39]:
result_df3['clean_message'] = result_df3['tweet'].apply(clean_text)
result_df3

,id,tweet,label,category,clean_message
0,1,The CDC currently reports 99031 deaths. In gen...,0,medical,cdc currently reports 99031 deaths general dis...
1,2,States reported 1121 deaths a small rise from ...,0,medical,states reported 1121 deaths small rise last tu...
2,3,Politically Correct Woman (Almost) Uses Pandem...,1,medical,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,0,medical,indiafightscorona 1524 covid testing laborator...
4,5,Populous states can generate large case counts...,0,medical,populous states generate large case counts loo...
...,...,...,...,...,...
8555,2136,Donald Trump wrongly claimed that New Zealand ...,1,medical,donald trump wrongly claimed new zealand “big ...
8556,2137,Current understanding is #COVID19 spreads most...,0,medical,current understanding covid19 spreads mostly p...
8557,2138,Nothing screams “I am sat around doing fuck al...,1,medical,nothing screams “i sat around fuck lockdown” q...
8558,2139,Birx says COVID-19 outbreak not under control ...,1,medical,birx says covid19 outbreak control ‘people mov...


# 끝

## Counter를 import하여 진짜 뉴스의 clean_message에 가장 빈도수가 높은 값 15개를 출력

In [40]:
from collections import Counter

common_columns = ['clean_message', 'label']
clean_message = pd.concat([result[common_columns], result_df[common_columns], result_df3[common_columns]], axis=0, ignore_index=True)

words = clean_message[clean_message['label']==0]['clean_message'].apply(lambda x: [word for word in x.split()])
true_news_words = Counter()

for msg in words:
    true_news_words.update(msg)

print(true_news_words.most_common(15))

[('said', 97577), ('trump', 44812), ('us', 44250), ('would', 32593), ('reuters', 28287), ('president', 26474), ('state', 20889), ('new', 19351), ('government', 18698), ('states', 17832), ('house', 17767), ('united', 16235), ('people', 16229), ('also', 16209), ('republican', 16125)]


## 가짜 뉴스의 clean_message에 가장 빈도수가 높은 값 15개를 출력

In [41]:
words = clean_message[clean_message['label']==1]['clean_message'].apply(lambda x: [word for word in x.split()])
true_news_words = Counter()

for msg in words:
    true_news_words.update(msg)

print(true_news_words.most_common(15))

[('trump', 79428), ('said', 31860), ('president', 27482), ('people', 27477), ('would', 24457), ('one', 24008), ('us', 23792), ('obama', 20320), ('clinton', 19259), ('like', 18227), ('donald', 17962), ('video', 15951), ('new', 15719), ('also', 15379), ('hillary', 15243)]


# 머신러닝 모델

In [42]:
clean_message

,clean_message,label
0,decline coal start started natural gas took st...,0
1,hillary clinton agrees john mccain voting give...,0
2,health care reform legislation likely mandate ...,1
3,economic turnaround started end term,0
4,chicago bears starting quarterbacks last 10 ye...,0
...,...,...
84170,donald trump wrongly claimed new zealand “big ...,1
84171,current understanding covid19 spreads mostly p...,0
84172,nothing screams “i sat around fuck lockdown” q...,1
84173,birx says covid19 outbreak control ‘people mov...,1


In [43]:
# clean_message = clean_message[clean_message['label'] != 2]
# clean_message['label'].value_counts()
# clean_message.to_csv('clean_message.csv', index=False)
# clean_message = pd.read_csv('clean_message.csv')
# clean_message

In [44]:
#학습 데이터와 검증 데이터로 나눌 train_test_split, naive bayes 모델인 MultinomialNB, clean_message에 있는 단어들의 특징 추출을 위해 문자열 데이터를 수치 벡터로 변환시켜주는 CountVectorizer를 import
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.utils import to_categorical

2023-12-19 16:46:30.574509: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-19 16:46:30.575507: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 16:46:30.591014: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 16:46:30.591311: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 16:46:30.990757: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [45]:
X = clean_message['clean_message']
y = clean_message['label']

## 전체 데이터에서 테스트 사이즈를 70%로 설정하고, random_state를 1234로 설정

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [47]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((58922,), (25253,), (58922,), (25253,))

## RNN

In [48]:
import tensorflow as tf
from tensorflow.keras import backend as K

class MatthewsCorrelationCoefficient(tf.keras.metrics.Metric):
    def __init__(self, name='mcc', **kwargs):
        super(MatthewsCorrelationCoefficient, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.true_negatives = self.add_weight(name='tn', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)
        y_true = tf.cast(y_true, tf.float32)  # y_true를 float32로 타입 캐스팅
        true_positives = K.sum(y_true * y_pred)
        true_negatives = K.sum((1 - y_true) * (1 - y_pred))
        false_positives = K.sum((1 - y_true) * y_pred)
        false_negatives = K.sum(y_true * (1 - y_pred))

        self.true_positives.assign_add(true_positives)
        self.true_negatives.assign_add(true_negatives)
        self.false_positives.assign_add(false_positives)
        self.false_negatives.assign_add(false_negatives)

    def result(self):
        numerator = (self.true_positives * self.true_negatives -
                     self.false_positives * self.false_negatives)
        denominator = K.sqrt((self.true_positives + self.false_positives) *
                             (self.true_positives + self.false_negatives) *
                             (self.true_negatives + self.false_positives) *
                             (self.true_negatives + self.false_negatives) + K.epsilon())

        return numerator / (denominator + K.epsilon())

    def reset_states(self):
        for s in [self.true_positives, self.false_positives, self.true_negatives, self.false_negatives]:
            K.set_value(s, 0.0)

In [49]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import tensorflow as tf
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU

# 텍스트 데이터를 숫자로 임베딩
max_words = 30000  # 최대 단어 수
max_sequence_length = 100  # 시퀀스 최대 길이

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

#RNN 모델 생성
model_RNN = Sequential()
model_RNN.add(Embedding(input_dim=max_words, output_dim=30, input_length=max_sequence_length))
model_RNN.add(SimpleRNN(100))
model_RNN.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_RNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=[MatthewsCorrelationCoefficient()])

# 모델 훈련
history_RNN = model_RNN.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))

# 모델 요약 출력
model_RNN.summary()

Epoch 1/10


2023-12-19 16:46:37.476509: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-19 16:46:37.476747: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1840/1842 [============================>.] - ETA: 0s - loss: 0.3626 - mcc: 0.6412

/home/t16user3/.conda/envs/example/lib/python3.9/site-packages/keras/src/engine/training.py:2620: UserWarning: Metric MatthewsCorrelationCoefficient implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


1842/1842 [==============================] - 22s 12ms/step - loss: 0.3625 - mcc: 0.6413 - val_loss: 0.2333 - val_mcc: 0.8145
Epoch 2/10
1842/1842 [==============================] - 21s 11ms/step - loss: 0.4415 - mcc: 0.5547 - val_loss: 0.4860 - val_mcc: 0.5039
Epoch 3/10
1842/1842 [==============================] - 20s 11ms/step - loss: 0.3173 - mcc: 0.7164 - val_loss: 0.2973 - val_mcc: 0.7406
Epoch 4/10
1842/1842 [==============================] - 22s 12ms/step - loss: 0.2113 - mcc: 0.8283 - val_loss: 0.2551 - val_mcc: 0.7939
Epoch 5/10
1842/1842 [==============================] - 21s 12ms/step - loss: 0.1654 - mcc: 0.8684 - val_loss: 0.2498 - val_mcc: 0.8048
Epoch 6/10
1842/1842 [==============================] - 22s 12ms/step - loss: 0.1125 - mcc: 0.9157 - val_loss: 0.2506 - val_mcc: 0.8301
Epoch 7/10
1842/1842 [==============================] - 21s 12ms/step - loss: 0.0819 - mcc: 0.9395 - val_loss: 0.2401 - val_mcc: 0.8546
Epoch 8/10
1842/1842 [==============================] - 22s

In [50]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import tensorflow as tf
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU

# 텍스트 데이터를 숫자로 임베딩
max_words = 30000  # 최대 단어 수
max_sequence_length = 100  # 시퀀스 최대 길이

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

#RNN 모델 생성
model_RNN = Sequential()
model_RNN.add(Embedding(input_dim=max_words, output_dim=30, input_length=max_sequence_length))
model_RNN.add(SimpleRNN(100))
model_RNN.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_RNN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
history_RNN = model_RNN.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))

# 모델 요약 출력
model_RNN.summary()

Epoch 1/10
1842/1842 [==============================] - 23s 12ms/step - loss: 0.3706 - accuracy: 0.8174 - val_loss: 0.2463 - val_accuracy: 0.8997
Epoch 2/10
1842/1842 [==============================] - 21s 11ms/step - loss: 0.2064 - accuracy: 0.9146 - val_loss: 0.2435 - val_accuracy: 0.8980
Epoch 3/10
1842/1842 [==============================] - 22s 12ms/step - loss: 0.1041 - accuracy: 0.9613 - val_loss: 0.1981 - val_accuracy: 0.9361
Epoch 4/10
1842/1842 [==============================] - 22s 12ms/step - loss: 0.0463 - accuracy: 0.9842 - val_loss: 0.2339 - val_accuracy: 0.9429
Epoch 5/10
1842/1842 [==============================] - 22s 12ms/step - loss: 0.0243 - accuracy: 0.9924 - val_loss: 0.2411 - val_accuracy: 0.9420
Epoch 6/10
1842/1842 [==============================] - 22s 12ms/step - loss: 0.0388 - accuracy: 0.9863 - val_loss: 0.2221 - val_accuracy: 0.9229
Epoch 7/10
1842/1842 [==============================] - 21s 11ms/step - loss: 0.0378 - accuracy: 0.9862 - val_loss: 0.2859 -

## LSTM

In [51]:
#LSTM 모델 생성
model_LSTM = Sequential()
model_LSTM.add(Embedding(input_dim=max_words, output_dim=30, input_length=max_sequence_length))
model_LSTM.add(Dropout(0.3))
model_LSTM.add(LSTM(100))
model_LSTM.add(Dropout(0.3))
model_LSTM.add(Dense(64, activation='relu'))
model_LSTM.add(Dropout(0.3))
model_LSTM.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_LSTM.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
history_LSTM = model_LSTM.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))

# 모델 요약 출력
model_LSTM.summary()

Epoch 1/10
1842/1842 [==============================] - 89s 48ms/step - loss: 0.3335 - accuracy: 0.8291 - val_loss: 0.2708 - val_accuracy: 0.8722
Epoch 2/10
1842/1842 [==============================] - 83s 45ms/step - loss: 0.2049 - accuracy: 0.9087 - val_loss: 0.2213 - val_accuracy: 0.9068
Epoch 3/10
1842/1842 [==============================] - 84s 45ms/step - loss: 0.1387 - accuracy: 0.9427 - val_loss: 0.2084 - val_accuracy: 0.9246
Epoch 4/10
1842/1842 [==============================] - 85s 46ms/step - loss: 0.0985 - accuracy: 0.9607 - val_loss: 0.2088 - val_accuracy: 0.9322
Epoch 5/10
1842/1842 [==============================] - 86s 46ms/step - loss: 0.0744 - accuracy: 0.9716 - val_loss: 0.2125 - val_accuracy: 0.9387
Epoch 6/10
1842/1842 [==============================] - 84s 46ms/step - loss: 0.0616 - accuracy: 0.9776 - val_loss: 0.2017 - val_accuracy: 0.9459
Epoch 7/10
1842/1842 [==============================] - 82s 45ms/step - loss: 0.0493 - accuracy: 0.9822 - val_loss: 0.2366 -

## GRU

In [52]:
#GRU 모델 생성
model_GRU = Sequential()
model_GRU.add(Embedding(input_dim=max_words, output_dim=64))
model_GRU.add(Dropout(0.3))
model_GRU.add(GRU(64))
model_GRU.add(Dropout(0.3))
model_GRU.add(Dense(16, activation='relu'))
model_GRU.add(Dropout(0.3))
model_GRU.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model_GRU.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
history_GRU = model_GRU.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))

# 모델 요약 출력
model_GRU.summary()

Epoch 1/10
1842/1842 [==============================] - 59s 31ms/step - loss: 0.3409 - accuracy: 0.8278 - val_loss: 0.2644 - val_accuracy: 0.8753
Epoch 2/10
1842/1842 [==============================] - 56s 31ms/step - loss: 0.1960 - accuracy: 0.9132 - val_loss: 0.2201 - val_accuracy: 0.9078
Epoch 3/10
1842/1842 [==============================] - 58s 32ms/step - loss: 0.1292 - accuracy: 0.9461 - val_loss: 0.2055 - val_accuracy: 0.9235
Epoch 4/10
1842/1842 [==============================] - 57s 31ms/step - loss: 0.0898 - accuracy: 0.9645 - val_loss: 0.2176 - val_accuracy: 0.9346
Epoch 5/10
1842/1842 [==============================] - 57s 31ms/step - loss: 0.0669 - accuracy: 0.9727 - val_loss: 0.2383 - val_accuracy: 0.9418
Epoch 6/10
1842/1842 [==============================] - 57s 31ms/step - loss: 0.0501 - accuracy: 0.9807 - val_loss: 0.2303 - val_accuracy: 0.9456
Epoch 7/10
1842/1842 [==============================] - 59s 32ms/step - loss: 0.0383 - accuracy: 0.9852 - val_loss: 0.2520 -